In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# import data
df = pd.read_csv('data.csv')

df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# remove latitude and longitude columns
df.drop(['Latitude'], axis=1, inplace=True)
df.drop(['Longitude'], axis=1, inplace=True)

In [ ]:
# find the number of missing values in each column
df.isnull().sum().sort_values(ascending=False)

In [ ]:
# remove the rows with missing values in the column 'Daily tests'
df = df.dropna(subset=['Daily tests']).reset_index(drop = True)

In [ ]:
df.isnull().sum().sort_values(ascending=False)

Removing countries with not enough data

In [ ]:
# find the sum of unique days in dataset
# total days between 1/1/2020 and 28/2/2021
total_days = 425 
total_countries = len(df['Entity'].unique())
# for each country, find the number of days 
countries = df['Entity'].unique()
for country in countries:
    # find the data frame for each country
    country_df = df[df['Entity'] == country]
    # find the number of days for each country
    country_days = country_df['Date'].nunique()
    # find the percent of days for each country
    percent = country_days / total_days
    if (percent<0.15): 
        # these countries have less than 15% of the days in the dataset
        # so we will remove them
        df.drop(df[df['Entity'] == country].index, inplace = True)
        # print(country, round(percent, 2))
# fix the index
df = df.reset_index(drop = True)
# print how many countries are left
print('Number of countries left: ',total_countries-len(df['Entity'].unique()))

Replace NaN values before first deaths and cases for each country with 0

In [ ]:
entityGroup = df.groupby('Entity')

for entity, group in entityGroup:
    for index in range(group.index[0], group.index[-1] + 1):
        if (df.isnull().iloc[index]['Cases']):
            df.at[index, 'Cases'] = 0
        else:
            break

for entity, group in entityGroup:
    for index in range(group.index[0], group.index[-1] + 1):
        if (df.isnull().iloc[index]['Deaths']):
            df.at[index, 'Deaths'] = 0
        else:
            break

df.isnull().sum().sort_values(ascending=False)

Convert total number of deaths and cases to daily number

In [ ]:
for entity, group in entityGroup:
    for index in range(group.index[0]+1, group.index[-1] + 1):
        daily_cases = df.at[index, 'Cases'] - df.at[index - 1, 'Cases'] 
        daily_deaths = df.at[index, 'Deaths'] - df.at[index - 1, 'Deaths']
        if (daily_cases < 0):
            df.at[index, 'Cases'] = df.at[index-1, 'Cases']
        if (daily_deaths < 0):
            df.at[index, 'Deaths'] = df.at[index-1, 'Deaths']

In [ ]:
# add a new column to the DataFrame for cases today
df['Daily cases'] = 0
# add a new column to the DataFrame for deaths today
df['Daily deaths'] = 0

dfCopy = df.copy()
for entity, group in entityGroup:
    for index in range(group.index[0]+1, group.index[-1] + 1):
        df.at[index, 'Daily cases'] = dfCopy.at[index, 'Cases'] - dfCopy.at[index - 1, 'Cases'] 
        df.at[index, 'Daily deaths'] = dfCopy.at[index, 'Deaths'] - dfCopy.at[index - 1, 'Deaths']

Drop the row where daily cases are more than daily tests

In [ ]:
df.drop(df[df['Daily tests'] < df['Daily cases']].index, inplace = True)
df = df.reset_index(drop = True)

Create Seasons Column\
1 (winter) -> December - February \
2 (spring) -> March - May \
3 (summer) -> June - August \
4 (autumn) -> September - November

In [ ]:
seasons = {1: 'Winter', 2: 'Spring', 3: 'Summer', 4: 'Autumn'}
    
df['Date'] =  pd.to_datetime(df['Date'])
df['Season'] = df['Date'].apply(lambda x: seasons[x.month%12 // 3 + 1])

Correlation Heatmap

In [ ]:
# Set the size of the figure
fig, ax = plt.subplots(figsize=(10, 10))

sns.heatmap(df.corr(numeric_only=True),annot=True,square=False, ax=ax)
plt.show()

Extract a csv file with the cleaned data

In [ ]:
# export datta to csv
df.to_csv('data_cleaned.csv', index=False)

In [ ]:
clusteringDf = df[['Entity', 'Daily tests', 'Daily cases', 'Daily deaths']].groupby('Entity').sum().reset_index()
clusteringDf['Population'] = df[['Entity', 'Population']].groupby('Entity')['Population'].first().reset_index().drop_duplicates('Entity')['Population']
clusteringDf['positivity_rate'] = (clusteringDf['Daily cases'] / clusteringDf['Daily tests']) * 100
clusteringDf['death_rate'] = (clusteringDf['Daily deaths'] / clusteringDf['Daily cases']) * 100
clusteringDf['mortality_rate'] = (clusteringDf['Daily cases'] / clusteringDf['Population']) * 100
clusteringDf

In [ ]:
from sklearn.cluster import KMeans
from collections import Counter

In [ ]:
distortion = []
for k in range(2,10):
    kmeans = KMeans(n_clusters=k,n_init=10)
    kmeans.fit(clusteringDf[['positivity_rate', 'death_rate', 'mortality_rate']])
    distortion.append(kmeans.inertia_)

plt.plot(range(2, 10), distortion, marker = 'X' )
plt.xlabel('Number of clusters')
plt.ylabel('SSE')
plt.show()

In [ ]:
from yellowbrick.cluster import  SilhouetteVisualizer
import numpy as np

for k in range(2,10):
    kmeans = KMeans(n_clusters=k,n_init=10)
    sil_vis = SilhouetteVisualizer(kmeans)
    sil_vis.fit(clusteringDf[['positivity_rate', 'death_rate', 'mortality_rate']])
    sil_vis.show()
    test = KMeans(n_clusters=k,n_init=10)
    test.fit(clusteringDf[['positivity_rate', 'death_rate', 'mortality_rate']])
    print(len(set(test.labels_)))

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
resultDf = clusteringDf[['positivity_rate', 'death_rate', 'mortality_rate']]
for k in range(2,6):
    kmeans = KMeans(n_clusters=k,n_init=10)
    label = kmeans.fit_predict(clusteringDf[['positivity_rate', 'death_rate', 'mortality_rate']])

    labels = kmeans.labels_
    colors = ['red', 'green', 'blue','yellow','black','magenda']
    resultDf = resultDf.copy()
    resultDf.loc[:, 'Graph_Color'] = [colors[label] for label in labels]
    ax = plt.axes(projection ='3d')
    ax.scatter(resultDf['positivity_rate'], resultDf['death_rate'], resultDf['mortality_rate'], c=resultDf['Graph_Color'])

    centroids = kmeans.cluster_centers_
    cluster_count=Counter(label)
    
    ax.scatter(centroids[:, 0], centroids[:, 1], centroids[:, 2], marker='x', s=200, linewidths=1, color='black')

    # Add labels to the plot
    ax.set_xlabel('Positivity rate')
    ax.set_ylabel('Death rate')
    ax.set_zlabel('Mortality rate')
    ax.set_title(f'{k} clusters with K-Means')

    # Show the plot
    plt.show()

    for i in range(k):
        print('cluster', i+1, 'with:', cluster_count[i], 'entities')
        clusterDf = df.groupby('Entity').describe()[kmeans.labels_==i].loc[:, (slice(None), ['min', 'max', 'mean'])]
        clusterDf2 = clusterDf.copy()
        df3 = clusteringDf[kmeans.labels_==i][['Entity', 'positivity_rate', 'death_rate', 'mortality_rate']].reset_index().drop('index', axis=1)
        df3= df3.set_index('Entity')
        clusterDf = clusterDf.drop(['Cases', 'Deaths', 'Daily tests', 'Daily cases', 'Daily deaths'], axis=1, level=0)
        displayDataframe = pd.concat([clusterDf.xs('mean', level=1, axis=1), clusterDf2[['Daily tests', 'Daily cases', 'Daily deaths']]], axis=1)
        displayDataframe = pd.concat([displayDataframe, df3[['positivity_rate', 'death_rate', 'mortality_rate']]], axis=1)
        # display(displayDataframe) # if we want to display statistics for each country in the cluster
        display(displayDataframe.describe().loc[['min', 'mean', 'max']])